In [2]:
!pip -q install torch torchvision torchmetrics==1.3.2 pillow tqdm nltk einops numpy==1.26.4

In [3]:
import os, json, math, random
from dataclasses import dataclass, asdict
from collections import Counter
from typing import List, Dict, Tuple
from contextlib import nullcontext
from tqdm import tqdm

import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch.nn.utils.rnn import pack_padded_sequence, pad_sequence
from torchmetrics.text.bleu import BLEUScore
import nltk
nltk.download('punkt', quiet=True)

True

#  Device & AMP setup  


In [5]:
from contextlib import nullcontext
import torch

def pick_device():
    if torch.backends.mps.is_available():
        return torch.device("mps")
    if torch.cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")

DEVICE = pick_device()
print("Using device:", DEVICE)

# new-style AMP (CUDA only)
amp_ctx = (lambda: torch.amp.autocast(device_type="cuda")) if DEVICE.type == "cuda" else (lambda: nullcontext())
scaler = torch.amp.GradScaler(device_type="cuda") if DEVICE.type == "cuda" else None
use_scaler = scaler is not None



Using device: mps


In [6]:
@dataclass
class Config:
    DATA_ROOT: str = "data"
    IMAGES_DIR: str = "data/Images"
    CAPTIONS_FILE: str = "data/captions.txt"  
    OUTPUT_DIR: str = "artifacts"

    IMG_SIZE: int = 224
    BATCH_SIZE: int = 64
    NUM_WORKERS: int = 0  # If you see Mac issues, set to 0
    EPOCHS: int = 12
    LR: float = 3e-4

    EMBED_DIM: int = 256
    HIDDEN_DIM: int = 512
    NUM_LAYERS: int = 1
    DROPOUT: float = 0.1

    MIN_FREQ: int = 3
    MAX_LEN: int = 28
    FREEZE_CNN: bool = True

    SEED: int = 42

cfg = Config()
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
with open(os.path.join(cfg.OUTPUT_DIR, "config.json"), "w") as f:
    json.dump(asdict(cfg), f, indent=2)
cfg


Config(DATA_ROOT='data', IMAGES_DIR='data/Images', CAPTIONS_FILE='data/captions.txt', OUTPUT_DIR='artifacts', IMG_SIZE=224, BATCH_SIZE=64, NUM_WORKERS=0, EPOCHS=12, LR=0.0003, EMBED_DIM=256, HIDDEN_DIM=512, NUM_LAYERS=1, DROPOUT=0.1, MIN_FREQ=3, MAX_LEN=28, FREEZE_CNN=True, SEED=42)

# Data + Vocab

In [8]:
SPECIALS = {"<pad>":0, "<bos>":1, "<eos>":2, "<unk>":3}

def tokenize(text: str) -> List[str]:
    return nltk.word_tokenize(text.lower())

def load_captions(captions_file: str) -> Dict[str, List[str]]:
    """
    Supports:
      - Flickr8k.token.txt       -> 'image.jpg#i\\tcaption'
      - captions.csv/txt (Kaggle)-> 'image,caption' (with optional header)
    Returns: { "image.jpg": [cap1, cap2, ...], ... }
    """
    img2caps: Dict[str, List[str]] = {}
    with open(captions_file, "r", encoding="utf-8") as f:
        lines = [l.strip() for l in f if l.strip()]

    # Detect token format vs CSV and CSV header
    is_token = ("\t" in lines[0]) and ("#" in lines[0].split("\t")[0])
    has_header = (not is_token) and lines[0].lower().startswith("image,")
    start_idx = 1 if has_header else 0

    for line in lines[start_idx:]:
        if is_token:
            left, cap = line.split("\t", 1)
            img = left.split("#")[0].strip()
        else:
            # CSV-like: split only on the first comma
            parts = line.split(",", 1)
            if len(parts) != 2:
                continue
            img, cap = parts[0].strip(), parts[1].strip()

        # remove wrapping quotes if present
        if img.startswith('"') and img.endswith('"'): img = img[1:-1]
        if cap.startswith('"') and cap.endswith('"'): cap = cap[1:-1]

        if img:
            img2caps.setdefault(img, []).append(cap)
    return img2caps

def filter_existing(img2caps: Dict[str, List[str]], images_dir: str) -> Dict[str, List[str]]:
    """Keep only entries whose image file exists in images_dir."""
    out = {}
    missing = 0
    for img, caps in img2caps.items():
        path = os.path.join(images_dir, img)
        if os.path.exists(path):
            out[img] = caps
        else:
            missing += 1
    if missing:
        print(f"[warn] Skipped {missing} caption entries with missing image files.")
    return out

def build_vocab(img2caps: Dict[str, List[str]], min_freq: int):
    counter = Counter()
    for caps in img2caps.values():
        for c in caps:
            counter.update(tokenize(c))
    # specials first
    itos = [None]*len(SPECIALS)
    for tok, idx in SPECIALS.items():
        itos[idx] = tok
    # add tokens by frequency
    for tok, freq in counter.items():
        if freq >= min_freq:
            itos.append(tok)
    stoi = {tok:i for i, tok in enumerate(itos)}
    return stoi, itos

# ---- Run the pipeline ----
raw_caps = load_captions(cfg.CAPTIONS_FILE)
raw_caps = filter_existing(raw_caps, cfg.IMAGES_DIR)

# (Optional sanity checks)
print("Images dir exists:", os.path.isdir(cfg.IMAGES_DIR))
example_key = next(iter(raw_caps)) if raw_caps else None
print("Example image key:", example_key)
if example_key:
    print("Example file exists:", os.path.exists(os.path.join(cfg.IMAGES_DIR, example_key)))

stoi, itos = build_vocab(raw_caps, cfg.MIN_FREQ)

# save vocab
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
with open(os.path.join(cfg.OUTPUT_DIR, "vocab.json"), "w", encoding="utf-8") as f:
    json.dump({"stoi": stoi, "itos": itos}, f)

print(f"Vocab size: {len(stoi)}")


Images dir exists: True
Example image key: 1000268201_693b08cb0e.jpg
Example file exists: True
Vocab size: 4108


# Dataset & Loaders

In [10]:
class Flickr8kDataset(Dataset):
    def __init__(self, images_dir, img2caps, transform, stoi, max_len=28, split="train", seed=42):
        self.images_dir = images_dir
        self.transform = transform
        self.stoi = stoi
        self.max_len = max_len

        imgs = list(img2caps.keys())
        random.Random(seed).shuffle(imgs)
        n = len(imgs)
        self.imgs = imgs[:int(0.8*n)] if split=="train" else imgs[int(0.8*n):int(0.9*n)] if split=="val" else imgs[int(0.9*n):]
        self.img2caps = img2caps

    def __len__(self): return len(self.imgs)

    def __getitem__(self, idx):
        img_name = self.imgs[idx]
        image = Image.open(os.path.join(self.images_dir, img_name)).convert("RGB")
        cap = random.choice(self.img2caps[img_name])

        image = self.transform(image) if self.transform else image
        tokens = ["<bos>"] + tokenize(cap)[: self.max_len - 2] + ["<eos>"]
        ids = [self.stoi.get(t, SPECIALS["<unk>"]) for t in tokens]
        return image, torch.tensor(ids, dtype=torch.long), img_name

def collate_fn(batch):
    imgs, seqs, names = zip(*batch)
    imgs = torch.stack(imgs)
    lengths = torch.tensor([len(s) for s in seqs], dtype=torch.long)
    seqs_padded = pad_sequence(seqs, batch_first=True, padding_value=SPECIALS["<pad>"])
    return imgs, seqs_padded, lengths, names

train_tfms = transforms.Compose([
    transforms.Resize((cfg.IMG_SIZE, cfg.IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])
eval_tfms = transforms.Compose([
    transforms.Resize((cfg.IMG_SIZE, cfg.IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

train_ds = Flickr8kDataset(cfg.IMAGES_DIR, raw_caps, train_tfms, stoi, cfg.MAX_LEN, "train", cfg.SEED)
val_ds   = Flickr8kDataset(cfg.IMAGES_DIR, raw_caps, eval_tfms,  stoi, cfg.MAX_LEN, "val",   cfg.SEED)
test_ds  = Flickr8kDataset(cfg.IMAGES_DIR, raw_caps, eval_tfms,  stoi, cfg.MAX_LEN, "test",  cfg.SEED)

train_loader = DataLoader(train_ds, batch_size=cfg.BATCH_SIZE, shuffle=True,  num_workers=cfg.NUM_WORKERS, collate_fn=collate_fn)
val_loader   = DataLoader(val_ds,   batch_size=cfg.BATCH_SIZE, shuffle=False, num_workers=cfg.NUM_WORKERS, collate_fn=collate_fn)
test_loader  = DataLoader(test_ds,  batch_size=cfg.BATCH_SIZE, shuffle=False, num_workers=cfg.NUM_WORKERS, collate_fn=collate_fn)

len(train_ds), len(val_ds), len(test_ds)


(6472, 809, 810)

In [11]:
class EncoderCNN(nn.Module):
    def __init__(self, embed_dim=256, freeze=True):
        super().__init__()
        base = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        self.backbone = nn.Sequential(*list(base.children())[:-1])  # remove FC
        self.proj = nn.Linear(base.fc.in_features, embed_dim)
        if freeze:
            for p in self.backbone.parameters(): p.requires_grad = False

    def forward(self, x):
        feats = self.backbone(x).flatten(1)  # (B,C)
        return self.proj(feats)              # (B,E)

class DecoderRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers=1, dropout=0.1):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=SPECIALS["<pad>"])
        self.lstm  = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True,
                             dropout=(dropout if num_layers>1 else 0.0))
        self.fc    = nn.Linear(hidden_dim, vocab_size)

    def forward(self, features, captions, lengths):
        x_tok = self.embed(captions)                    # (B,T,E)
        x_img = features.unsqueeze(1)                   # (B,1,E)
        x     = torch.cat([x_img, x_tok[:, :-1, :]], 1) # prepend image, shift caps
        packed = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        out, _ = self.lstm(packed)
        logits = self.fc(out.data)
        return logits

    @torch.no_grad()
    def greedy(self, features, max_len=28):
        B = features.size(0)
        h = None
        token = torch.full((B,1), SPECIALS["<bos>"], dtype=torch.long, device=features.device)
        x = self.embed(token)                           # (B,1,E)
        x = torch.cat([features.unsqueeze(1), x], dim=1)[:, -1:, :]
        outs = []
        for _ in range(max_len):
            y, h = self.lstm(x, h)
            logits = self.fc(y.squeeze(1))
            nxt = torch.argmax(logits, -1, keepdim=True)  # (B,1)
            outs.append(nxt)
            x = self.embed(nxt)
        return torch.cat(outs, 1)

class CaptionNet(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers, dropout, freeze_cnn=True):
        super().__init__()
        self.enc = EncoderCNN(embed_dim, freeze=freeze_cnn)
        self.dec = DecoderRNN(vocab_size, embed_dim, hidden_dim, num_layers, dropout)

    def forward(self, images, captions, lengths):
        feats = self.enc(images)
        return self.dec(feats, captions, lengths)

    @torch.no_grad()
    def generate(self, images, max_len=28):
        feats = self.enc(images)
        return self.dec.greedy(feats, max_len)


In [12]:
def ids_to_text(ids, itos):
    words = []
    for i in ids:
        w = itos[int(i)]
        if w == "<eos>": break
        if w in {"<bos>","<pad>"}: continue
        words.append(w)
    return " ".join(words)
    
def compute_bleu(preds: list[str], refs: list[list[str]]) -> float:
    """
    preds: ['a dog runs...', ...]
    refs:  [['a dog running...', 'a canine...', ...], [...], ...]
    BLEUScore will tokenize internally, so pass strings.
    """
    bleu = BLEUScore(n_gram=4, smooth=True)
    score = bleu(preds, refs)  # returns a 0-dim tensor
    return float(score.cpu().item())

def evaluate(model, loader, criterion, itos, device, max_len):
    model.eval()
    total_loss, n_tok = 0.0, 0
    preds, refs = [], []

    with torch.no_grad():
        for images, caps, lengths, names in loader:
            images, caps = images.to(device), caps.to(device)

            # compute loss
            logits = model(images, caps, lengths)
            targets = pack_padded_sequence(caps, lengths, batch_first=True, enforce_sorted=False)[0]
            loss = criterion(logits, targets)
            total_loss += loss.item() * targets.size(0)
            n_tok += targets.size(0)

            # decode predictions
            gen = model.generate(images, max_len=max_len)
            for i in range(images.size(0)):
                pred_txt = ids_to_text(gen[i], itos)      # string
                # use ALL refs for this image as strings (no tokenization here)
                all_refs = raw_caps[names[i]]             # list[str], e.g., 5 captions
                preds.append(pred_txt)
                refs.append(all_refs)

    bleu = compute_bleu(preds, refs) if preds else 0.0
    ppl  = math.exp(total_loss / max(1, n_tok))
    return total_loss/max(1,n_tok), bleu, ppl


In [21]:
torch.manual_seed(cfg.SEED); random.seed(cfg.SEED); np.random.seed(cfg.SEED)

vocab_size = len(stoi)
model = CaptionNet(
    vocab_size=vocab_size,
    embed_dim=cfg.EMBED_DIM,
    hidden_dim=cfg.HIDDEN_DIM,
    num_layers=cfg.NUM_LAYERS,
    dropout=cfg.DROPOUT,
    freeze_cnn=cfg.FREEZE_CNN,
).to(DEVICE)

criterion = nn.CrossEntropyLoss(ignore_index=SPECIALS["<pad>"]).to(DEVICE)
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=cfg.LR)
scaler = torch.amp.GradScaler(device_type="cuda") if DEVICE.type == "cuda" else None
use_scaler = scaler is not None

best_bleu, save_path = 0.0, os.path.join(cfg.OUTPUT_DIR, "weights.pt")

for epoch in range(1, cfg.EPOCHS+1):
    print(f"\n=== Epoch {epoch}/{cfg.EPOCHS} ===")
    model.train()
    pbar = tqdm(train_loader, desc=f"Training")
    run_loss, tok = 0.0, 0

    for batch_idx, (images, caps, lengths, names) in enumerate(pbar, start=1):
        images, caps = images.to(DEVICE), caps.to(DEVICE)
        optimizer.zero_grad(set_to_none=True)

        with amp_ctx():
            logits = model(images, caps, lengths)
            targets = pack_padded_sequence(caps, lengths, batch_first=True, enforce_sorted=False)[0]
            loss = criterion(logits, targets)

        if use_scaler:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        run_loss += loss.item() * targets.size(0)
        tok += targets.size(0)
        avg_loss = run_loss / max(1, tok)

        # update tqdm bar
        pbar.set_postfix(loss=f"{avg_loss:.4f}")

        # print batch info every 10 batches
        if batch_idx % 10 == 0 or batch_idx == 1:
            print(f"  [Epoch {epoch} | Batch {batch_idx}/{len(train_loader)}] "
                  f"Loss: {loss.item():.4f} | Running Avg: {avg_loss:.4f}")

    # ---- End of epoch validation ----
    vloss, vbleu, vppl = evaluate(model, val_loader, criterion, itos, DEVICE, cfg.MAX_LEN)
    print(f"End of Epoch {epoch}: Val loss {vloss:.4f} | BLEU4 {vbleu:.4f} | PPL {vppl:.2f}")

    if vbleu > best_bleu:
        best_bleu = vbleu
        torch.save({"model": model.state_dict(), "vocab_size": vocab_size}, save_path)
        print(f"  ✓ New best BLEU {best_bleu:.4f} — saved checkpoint to {save_path}")



=== Epoch 1/12 ===


Training:   1%|▏                   | 1/102 [00:00<01:17,  1.30it/s, loss=8.3245]

  [Epoch 1 | Batch 1/102] Loss: 8.3245 | Running Avg: 8.3245


Training:  10%|█▊                 | 10/102 [00:05<00:41,  2.24it/s, loss=8.1278]

  [Epoch 1 | Batch 10/102] Loss: 7.8802 | Running Avg: 8.1278


...

  [Epoch 12 | Batch 90/102] Loss: 3.0035 | Running Avg: 2.8862


Training:  98%|█████████████████▋| 100/102 [00:40<00:00,  2.43it/s, loss=2.8907]

  [Epoch 12 | Batch 100/102] Loss: 3.0938 | Running Avg: 2.8907


Training: 100%|██████████████████| 102/102 [00:41<00:00,  2.48it/s, loss=2.8908]


End of Epoch 12: Val loss 2.9781 | BLEU4 0.0576 | PPL 19.65


In [23]:
# ---- Load best checkpoint and evaluate on test set ----
ckpt = torch.load(save_path, map_location="cpu")
model.load_state_dict(ckpt["model"])
model = model.to(DEVICE).eval()

t_loss, t_bleu, t_ppl = evaluate(model, test_loader, criterion, itos, DEVICE, cfg.MAX_LEN)
print(f"\n=== TEST RESULTS ===")
print(f"Loss: {t_loss:.4f} | BLEU4: {t_bleu:.4f} | Perplexity: {t_ppl:.2f}")

# ---- Show a few generated samples ----
model.eval()
with torch.no_grad():
    for images, caps, lengths, names in test_loader:
        images, caps = images.to(DEVICE), caps.to(DEVICE)
        gen = model.generate(images, max_len=cfg.MAX_LEN)

        for i in range(min(5, images.size(0))):
            pred_caption = ids_to_text(gen[i], itos)
            ref_caption  = ids_to_text(caps[i], itos)
            print(f"[{names[i]}]\n  pred: {pred_caption}\n  ref:  {ref_caption}\n")
        break


/var/folders/tz/h_6bmgm14l1c409h00871rp40000gn/T/ipykernel_13187/3822447336.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(save_path, map_location="cp


=== TEST RESULTS ===
Loss: 3.5325 | BLEU4: 0.0705 | Perplexity: 34.21
[3153067758_53f003b1df.jpg]
  pred: a man in a red shirt is running in the grass .
  ref:  a woman sitting on a bus with a paper bag hanging on a carrier

[3449170348_34dac4a380.jpg]
  pred: a man in a red shirt is running in the grass .
  ref:  a girl dances on a sidewalk .

[3626964430_cb5c7e5acc.jpg]
  pred: a man in a red shirt is running in the grass .
  ref:  people playing cricket in the park , pine trees in the back .

[2286823363_7d554ea740.jpg]
  pred: a man in a red shirt is running in the grass .
  ref:  a young boy jumping from one chair to another in his house

[241347204_007d83e252.jpg]
  pred: a man in a red shirt is running in the grass .
  ref:  football players gather around the <unk> .



In [25]:
# ---- Reload model on CPU for export ----
export_model = CaptionNet(
    vocab_size=vocab_size,
    embed_dim=cfg.EMBED_DIM,
    hidden_dim=cfg.HIDDEN_DIM,
    num_layers=cfg.NUM_LAYERS,
    dropout=cfg.DROPOUT,
    freeze_cnn=cfg.FREEZE_CNN,
).cpu()
export_model.load_state_dict(torch.load(save_path, map_location="cpu")["model"])
export_model.eval()

class InferenceWrapper(nn.Module):
    def __init__(self, net: CaptionNet, max_len: int):
        super().__init__()
        self.net = net
        self.max_len = max_len
    def forward(self, images: torch.Tensor):
        return self.net.generate(images, max_len=self.max_len)

wrapper = InferenceWrapper(export_model, cfg.MAX_LEN).cpu().eval()
dummy = torch.randn(1,3,cfg.IMG_SIZE,cfg.IMG_SIZE, device="cpu")

# ---- TorchScript export ----
ts_path = os.path.join(cfg.OUTPUT_DIR, "model_ts.pt")
ts = torch.jit.trace(wrapper, dummy)
torch.jit.save(ts, ts_path)
print(f"Saved TorchScript: {ts_path}")

# ---- ONNX export ----
onnx_path = os.path.join(cfg.OUTPUT_DIR, "model.onnx")
try:
    torch.onnx.export(
        wrapper, dummy, onnx_path,
        input_names=["images"], output_names=["token_ids"],
        opset_version=14,
        dynamic_axes={"images": {0: "batch"}, "token_ids": {0: "batch"}}
    )
    print(f"Saved ONNX: {onnx_path}")
except Exception as e:
    print("ONNX export skipped:", e)

# ---- Write artifact readme ----
with open(os.path.join(cfg.OUTPUT_DIR, "artifact_README.txt"), "w") as f:
    f.write("Input: float32 (B,3,224,224) normalized to ImageNet mean/std.\n")
    f.write("Output: int64 token IDs (B,T). Decode with vocab.json (itos).\n")

print("Artifacts:", os.listdir(cfg.OUTPUT_DIR))


/var/folders/tz/h_6bmgm14l1c409h00871rp40000gn/T/ipykernel_13187/655067795.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  export_model.load_state_dict(torch.load(save_

Saved TorchScript: artifacts/model_ts.pt


/opt/anaconda3/lib/python3.12/site-packages/torch/onnx/symbolic_opset9.py:4279: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


ONNX export skipped: Module onnx is not installed!
Artifacts: ['model_ts.pt', 'config.json', 'weights.pt', 'vocab.json', 'artifact_README.txt']
